## Content-Based Filtering model

Content-based filtering approaches leverage description or attributes from items the user has interacted to recommend similar items. It depends only on the user previous choices, making this method robust to avoid the cold-start problem. For textual items, like articles, news and books, it is simple to use the raw text to build item profiles and user profiles. Here we are using a very popular technique in information retrieval (search engines) named TF-IDF. This technique converts unstructured text into a vector structure, where each word is represented by a position in the vector, and the value measures how relevant a given word is for an article. As all items will be represented in the same Vector Space Model, it is to compute similarity between articles.

In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
import matplotlib.pyplot as plt

In [47]:
articles_df = pd.read_csv('C:\\Users\\ashok.kumar\\Documents\\rec systems\\shared_articles.csv\\shared_articles.csv')
articles_df = articles_df[articles_df['eventType'] == 'CONTENT SHARED']
print(len(articles_df['title']))
articles_df = articles_df.drop_duplicates(subset=['title'],keep= 'first')
print(len(articles_df['title']))

3047
3003


In [48]:
interactions_df = pd.read_csv('C:/Users/ashok.kumar/Documents/rec systems/users_interactions.csv/users_interactions.csv')
# interactions_df.head(10)

In [49]:
event_type_strength = {
    'VIEW' : 1.0,
    'FOLLOW' : 2.0,
    'BOOKMARK' : 2.5,
    'LIKE' : 3.0,
    'COMMENT CREATED' : 4.0,
}

In [50]:
# event_type_strength.keys(), event_type_strength.values()
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x : event_type_strength[x])
# interactions_df.head(5)

In [51]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()

In [52]:
users_interactions_count_df = interactions_df.groupby(['personId', 'contentId']).size().groupby('personId').size()
# print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[['personId']]
# print('# users with at least 5 interactions: %d' % len(users_with_enough_interactions_df))

In [53]:
# print('#number of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'personId',
               right_on = 'personId')
# print('# number of interactions from users with at least 5 interactions: %d'
#       % len(interactions_from_selected_users_df))

In [54]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
# print('#number of unique user/item interactions: %d' % len(interactions_full_df))
# interactions_full_df.head(10)

In [55]:
#Ignoring stopwords (words with no semantics) from English and Portuguese (as we have a corpus with mixed languages)
stopwords_list = stopwords.words('english') + stopwords.words('portuguese')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer='word',
                     ngram_range=(1, 2),
                     min_df=0.003,
                     max_df=0.5,
                     max_features=5000,
                     stop_words=stopwords_list)

item_ids = articles_df['contentId'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['title'] + "" + articles_df['text'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

<3003x5000 sparse matrix of type '<class 'numpy.float64'>'
	with 627188 stored elements in Compressed Sparse Row format>

In [56]:
def get_item_profile(item_id):
    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):
    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[person_id]
    user_item_profiles = get_item_profiles(interactions_person_df['contentId'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_full_df[interactions_full_df['contentId'] \
                                                   .isin(articles_df['contentId'])].set_index('personId')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [57]:
user_profiles = build_users_profiles()
len(user_profiles)

1140

In [58]:
myprofile = user_profiles[-1479311724257856983]
print(myprofile.shape)
pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[-1479311724257856983].flatten().tolist()), key=lambda x: -x[1])[:20],
             columns=['token', 'relevance'])

(1, 5000)


,token,relevance
0,learning,0.293460
1,machine learning,0.247992
2,machine,0.239613
3,google,0.207207
4,data,0.172910
5,ai,0.141888
6,algorithms,0.103226
7,graph,0.101202
8,like,0.098724
9,language,0.086874


In [61]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=1000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['contentId', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'contentId', 
                                                          right_on = 'contentId')[['contentId','recStrength', 'title']]
            return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

In [62]:
content_based_recommender_model.recommend_items(-1479311724257856983, topn=10, verbose=True)

,recStrength,title
0,0.665981,"How Google is Remaking Itself as a ""Machine Le..."
1,0.611802,Machine Learning for Designers
2,0.579658,Machine Learning Is No Longer Just for Experts
3,0.566454,How real businesses are using machine learning
4,0.562318,Building AI Is Hard-So Facebook Is Building AI...
5,0.557759,5 Skills You Need to Become a Machine Learning...
6,0.554625,Machine Learning as a Service: How Data Scienc...
7,0.552082,Is machine learning the next commodity?
8,0.544568,Google's Cloud Machine Learning service is now...
9,0.526078,Clarifying the uses of artificial intelligence...
